In [1]:
import pandas as pd
import os
import glob

# Define the folder paths containing the CSV files
folder_names = ['energy', 'regulation', 'reserve']

# Create a dictionary to store concatenated dataframes by folder
data = {}

# Iterate over each folder path
for folder_name in folder_names:
    # Get a list of all CSV files in the folder
    folder_path = 'data/prices/' + folder_name + "/"
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

    # Check if files are found
    if not csv_files:
        print(f"No CSV files found in the directory: {folder_path}")
        continue

    print(f"Found {len(csv_files)} CSV files in {folder_path}")

    # Create a list to store dataframes
    dataframes = []

    # Iterate over the list of CSV files
    for file in csv_files:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file)
        
        # Append the dataframe to the list
        dataframes.append(df)

    # Concatenate all dataframes into a single dataframe for the current folder
    concatenated_df = pd.concat(dataframes, ignore_index=True)

    # Store the concatenated dataframe in the dictionary
    data[folder_name] = concatenated_df



Found 12 CSV files in data/prices/energy/
Found 12 CSV files in data/prices/regulation/
Found 12 CSV files in data/prices/reserve/


In [2]:
# Create a dictionary to map periods to specific times
period_to_time = {
    1: "00:00",  2: "00:30", 3: "01:00", 4: "01:30", 5: "02:00",     6: "02:30",
    7: "03:00",    8: "03:30",    9: "04:00",    10: "04:30",
    11: "05:00",    12: "05:30",    13: "06:00",    14: "06:30",
    15: "07:00",    16: "07:30",    17: "08:00",    18: "08:30",
    19: "09:00",    20: "09:30",    21: "10:00",    22: "10:30",
    23: "11:00",    24: "11:30",    25: "12:00",    26: "12:30",
    27: "13:00",    28: "13:30",    29: "14:00",    30: "14:30",
    31: "15:00",    32: "15:30",    33: "16:00",    34: "16:30",
    35: "17:00",    36: "17:30",    37: "18:00",    38: "18:30",
    39: "19:00",    40: "19:30",    41: "20:00",    42: "20:30",
    43: "21:00",    44: "21:30",    45: "22:00",    46: "22:30",
    47: "23:00",    48: "23:30"
}


In [3]:
energy_price_df = data['energy'][['DATE', 'PERIOD', 'USEP ($/MWh)', 'LCP ($/MWh)']]
energy_price_df.columns = ['date', 'period', 'energy_price', 'dr_price']

# Ensure we are working with the original dataframe
energy_price = energy_price_df.copy()

# Map the periods to times
energy_price.loc[:, "timestep"] = energy_price["period"].map(period_to_time)

# Concatenate the date and time columns to create new labels
energy_price.loc[:, "Time"] = energy_price["date"] + " " + energy_price["timestep"]

energy_price = energy_price[['Time', 'energy_price', 'dr_price']].set_index(['Time'])

energy_price

,energy_price,dr_price
Time,,
01-Apr-2023 00:00,156.64,0.0
01-Apr-2023 00:30,203.80,0.0
01-Apr-2023 01:00,167.31,0.0
01-Apr-2023 01:30,177.07,0.0
01-Apr-2023 02:00,156.56,0.0
...,...,...
30-Sep-2023 21:30,128.40,0.0
30-Sep-2023 22:00,128.60,0.0
30-Sep-2023 22:30,128.40,0.0


In [4]:
regulation_price = data['regulation'][['DATE', 'PERIOD', 'PRICE ($/MWh)']]
regulation_price.columns = ['date', 'period', 'reg_price']

regulation_price.loc[:, "timestep"] = regulation_price["period"].map(period_to_time)

regulation_price.loc[:, "Time"] = regulation_price["date"] + " " + regulation_price["timestep"]


regulation_price = regulation_price[['Time', 'reg_price']].set_index(['Time'])

regulation_price

,reg_price
Time,
01-Apr-2023 00:00,37.00
01-Apr-2023 00:30,15.00
01-Apr-2023 01:00,10.00
01-Apr-2023 01:30,1.33
01-Apr-2023 02:00,1.33
...,...
30-Sep-2023 21:30,10.03
30-Sep-2023 22:00,10.00
30-Sep-2023 22:30,10.00


In [5]:
reserve_price = data['reserve'][['RESERVE GROUP', 'DATE', 'PERIOD', 'PRICE ($/MWh)']]
reserve_price.loc[:, "timestep"] = reserve_price["PERIOD"].map(period_to_time)
reserve_price.loc[:, "Time"] = reserve_price["DATE"] + " " + reserve_price["timestep"]
reserve_price = reserve_price[['RESERVE GROUP', 'Time', 'PRICE ($/MWh)']]
reserve_price

,RESERVE GROUP,Time,PRICE ($/MWh)
0,CONRESA,01-Apr-2023 00:00,0.10
1,CONRESB,01-Apr-2023 00:00,0.08
2,CONRESC,01-Apr-2023 00:00,0.07
3,CONRESD,01-Apr-2023 00:00,0.06
4,CONRESE,01-Apr-2023 00:00,0.02
...,...,...,...
169915,PRIRESA,30-Sep-2023 23:30,0.01
169916,PRIRESB,30-Sep-2023 23:30,0.01
169917,PRIRESC,30-Sep-2023 23:30,0.01
169918,PRIRESD,30-Sep-2023 23:30,0.01


In [6]:
cons_reserve_price = reserve_price[reserve_price["RESERVE GROUP"].str.contains("CON")]
cons_reserve_price.columns = ['Group', 'Time', 'cres_price']
cons_reserve_price = cons_reserve_price.set_index(['Group','Time'])
cons_reserve_price

,,cres_price
Group,Time,
CONRESA,01-Apr-2023 00:00,0.10
CONRESB,01-Apr-2023 00:00,0.08
CONRESC,01-Apr-2023 00:00,0.07
CONRESD,01-Apr-2023 00:00,0.06
CONRESE,01-Apr-2023 00:00,0.02
...,...,...
CONRESA,30-Sep-2023 23:30,0.02
CONRESB,30-Sep-2023 23:30,0.02
CONRESC,30-Sep-2023 23:30,0.01


In [7]:
prim_reserve_price = reserve_price[reserve_price["RESERVE GROUP"].str.contains("PRI")]
prim_reserve_price.columns = ['Group', 'Time', 'pres_price']
prim_reserve_price = prim_reserve_price.set_index(['Group','Time'])
prim_reserve_price

,,pres_price
Group,Time,
PRIRESA,01-Apr-2023 00:00,0.55
PRIRESB,01-Apr-2023 00:00,0.47
PRIRESC,01-Apr-2023 00:00,0.41
PRIRESD,01-Apr-2023 00:00,0.33
PRIRESE,01-Apr-2023 00:00,0.14
...,...,...
PRIRESA,30-Sep-2023 23:30,0.01
PRIRESB,30-Sep-2023 23:30,0.01
PRIRESC,30-Sep-2023 23:30,0.01


In [8]:
prim_category = 'PRIRESA'
cons_category = 'CONRESA'

merged_df = pd.merge(energy_price, regulation_price, on="Time")
merged_df = pd.merge(merged_df, prim_reserve_price.loc[prim_category], on="Time")
merged_df = pd.merge(merged_df, cons_reserve_price.loc[cons_category], on="Time")
merged_df


,energy_price,dr_price,reg_price,pres_price,cres_price
Time,,,,,
01-Apr-2023 00:00,156.64,0.0,37.00,0.55,0.10
01-Apr-2023 00:30,203.80,0.0,15.00,0.14,0.10
01-Apr-2023 01:00,167.31,0.0,10.00,0.01,0.01
01-Apr-2023 01:30,177.07,0.0,1.33,0.01,0.01
01-Apr-2023 02:00,156.56,0.0,1.33,0.01,0.01
...,...,...,...,...,...
30-Sep-2023 21:30,128.40,0.0,10.03,0.01,0.10
30-Sep-2023 22:00,128.60,0.0,10.00,0.01,0.05
30-Sep-2023 22:30,128.40,0.0,10.00,0.01,0.02


In [9]:
merged_df = merged_df.reset_index()
merged_df['Time'] = pd.to_datetime(merged_df['Time'])
merged_df = merged_df.set_index(['Time']).sort_index()
merged_df

,energy_price,dr_price,reg_price,pres_price,cres_price
Time,,,,,
2023-01-01 00:00:00,158.62,0.0,10.00,0.01,0.01
2023-01-01 00:30:00,147.58,0.0,1.20,0.01,0.01
2023-01-01 01:00:00,126.83,0.0,0.01,0.01,0.01
2023-01-01 01:30:00,144.53,0.0,0.01,0.01,0.01
2023-01-01 02:00:00,144.51,0.0,0.01,0.01,0.01
...,...,...,...,...,...
2023-12-20 21:30:00,120.02,0.0,15.09,0.01,0.10
2023-12-20 22:00:00,119.97,0.0,15.00,0.01,0.01
2023-12-20 22:30:00,117.53,0.0,15.00,0.01,0.01


In [10]:
with pd.ExcelWriter('data/prices/sg_prices.xlsx') as writer:
    for folder_name in folder_names:
        data[folder_name].to_excel(writer, sheet_name=folder_name, index=False)

    merged_df.to_excel('sg_prices_processed2.xlsx', float_format="%.20f")

In [11]:
from scripts.price_process import price_process
dir_path = 'data/prices/'
prim_category = 'PRIRESA'
cons_category = 'CONRESA'

price_df = price_process(dir_path, prim_category, cons_category)

Found 12 CSV files in data/prices/energy/
Found 12 CSV files in data/prices/regulation/
Found 12 CSV files in data/prices/reserve/
